<a href="https://colab.research.google.com/github/aniket-1812/MLDL/blob/master/datacleaning_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?
### Take this monstrosity as the DataFrame to use in the following puzzles:

```
df = pd.DataFrame({                  
                   'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                   'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                   'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67,32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']
})

```

In [5]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                   'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                   'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']
})

In [6]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


### 1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [16]:
for i in df["FlightNumber"].index:
  if pd.isnull(df["FlightNumber"][i]):
    df.loc[i, "FlightNumber"] = df.loc[i-1]["FlightNumber"]+10

In [17]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055.0,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075.0,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


### 2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [18]:
temp_df = pd.DataFrame()

In [23]:
temp_df[["From", "To"]] = df["From_To"].str.split("_", expand=True)

In [24]:
temp_df

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


### 3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [35]:
for col in temp_df.columns:
  temp_df[col] = temp_df[col].str.capitalize()

In [36]:
temp_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


### 4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [37]:
df.drop("From_To", axis=1, inplace=True)

In [39]:
df = df.join(temp_df, how="left")

In [40]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,10055.0,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075.0,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


### 5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.
### Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [55]:
new_df = pd.DataFrame(df["RecentDelays"].values.tolist())

In [61]:
col_dict = {}
col_list = []
for i in new_df.columns:
  key = i
  value = "Delay "+str(i+1)
  col_dict[key] = value

col_dict

{0: 'Delay 1', 1: 'Delay 2', 2: 'Delay 3'}

In [63]:
new_df.rename(columns=col_dict, inplace=True)

In [64]:
new_df

,Delay 1,Delay 2,Delay 3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [67]:
df.drop("RecentDelays",axis=1,inplace=True)

In [69]:
df = df.join(new_df)

In [70]:
df

,FlightNumber,Airline,From,To,Delay 1,Delay 2,Delay 3
0,10045.0,KLM(!),London,Paris,23.0,47.0,NaN
1,10055.0,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065.0,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075.0,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085.0,"""Swiss Air""",Brussels,London,67.0,32.0,NaN
